## Task 1. Stateful wordcount

In this task you're receiving batches in real time through DStream. You need to create the Wordcount program with the saving and updating of the state after each batch. 

You have to print the TOP-10 most popular words in the input dataset and its quantity. 

There are several points for this task:

1) You have to print the data only at the end. The criteria is if you have received the first empty RDD, the stream is finished. At this moment you have to print the result and stop the context.

2) You may split the line with using $flatMap$ method in DStream.

3) In this task you need to filter out short words (with length less than 4). For this aim you may use $filter$ method in DStream.

4) Remember that you should use string lowercase. You may use $map$ method in DStream to transform words to such case.

5) You may use  $reduceByKey$ in Dstream to merge the tuples with the same key by summing the word count value.

6) In this task, you need to be able to maintain the state across the batches. You may use the $updateStateByKey()$ method, which provides an access to the state variable and helps you to implement the "stateful" approach. You can update the current state with the results of every batch.

You may find more useful methods in the following sources:

* Book "Learning Spark: Lightning-Fast Big Data Analysis" by Holden Karau.

* [Spark Streaming documentation](https://spark.apache.org/docs/latest/streaming-programming-guide.html)

* [PySpark Streaming documentation](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark-streaming-module) 

* [PySpark Streaming examples](https://github.com/apache/spark/tree/master/examples/src/main/python/streaming)


Here you can find the starter for the main steps of the task. You can use other methods to get the solution.

In [1]:
import os
import re
import time
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

**NB.** Please don't change the cell below. It is used for emulation realtime batch arriving. But figure out the code, it will help you when you work with real SparkStreaming applications.

In [2]:
# Preparing SparkContext
sc = SparkContext(master='local[4]')

# Preparing batches with the input data
DATA_PATH = "/data/wiki/en_articles_streaming"

batches = [sc.textFile(os.path.join(DATA_PATH, path)) for path in os.listdir(DATA_PATH)]

# Creating Dstream to emulate realtime data generating
BATCH_TIMEOUT = 5  # Timeout between batch generation
ssc = StreamingContext(sc, BATCH_TIMEOUT)
dstream = ssc.queueStream(rdds=batches)

There are 2 flags used in this task.
* The `finished` flag indicates if the current RDD is empty.
* The `printed` one indicates that the result has been printed and SparkStreaming context can be stopped.

For filtering out punctuation and other junk symbols use this pattern: `re.split("\W*\s+\W*", line.strip(), flags=re.UNICODE)`

**NB**. Spark transformations work in a lazy mode. When the transformation is called, it doesn't execute really. It just saves in the computational DAG. All the transformations will be executed when the action will be called. Let's look at `print_only_at_the_end()` function. The action will be called only when the stream will be finished. So in this moment  Spark will execute all the transformations. This will lead to container's overflow if the dataset is really big. So if you faced the error like `Container killed by YARN for exceeding memory limits`, call some action before `if` clause in this function.

In [3]:
finished = False
printed = False


def set_ending_flag(rdd):
    global finished
    if rdd.isEmpty():
        finished = True


def print_only_at_the_end(rdd):
    global printed
    # Type your code for printing the sorted data from the stream in a loop
    rdd.count()
    if finished and not printed:
        words = rdd.sortBy(lambda a: -a[1]).take(10)
        
        for word in words:
            print('{} {}'.format(word[0], word[1]))
        printed = True

# If we have received an empty rdd, the stream is finished.
# So print the result and stop the context.
dstream.foreachRDD(set_ending_flag)

In [ ]:
# Type your code for data processing and aggregation here
def aggregator(values, old):
    return (old or 0) + sum(values)

dstream.map(lambda doc: doc.split('\t', 1)[1])\
                    .flatMap(lambda line: re.split("\W*\s+\W*", line.strip(), flags=re.UNICODE))\
                    .filter(lambda word: len(word) > 3)\
                    .map(lambda word: (word.lower(), 1))\
                    .reduceByKey(lambda x, y: x + y)\
                    .updateStateByKey(aggregator)\
                    .foreachRDD(print_only_at_the_end)


**NB.** Please don't change the cell below. It is used for stopping SparkStreaming context and Spark context when the stream finished.

In [ ]:
ssc.checkpoint('./checkpoint{}'.format(time.strftime("%Y_%m_%d_%H_%M_%s", time.gmtime())))  # checkpoint for storing current state        
ssc.start()
while not printed:
    pass
ssc.stop()  # when the result was printed, stop SparkStreaming context
sc.stop()  # stop Spark context to be able restart the code without restarting the kernel

that 81572
with 79559
from 58201
which 42198
this 38252
were 34403
also 31573
have 29871
their 24579
other 23538
first 22239
they 20572
been 20466
such 19602
more 18578
used 18233
some 18097
most 17610
american 17217
after 16169
into 15630
when 15425
there 14719
many 14640
than 14639
these 14631
only 14428
time 13567
between 12918
would 12789
during 12383
world 11492
over 11348
about 10932
years 10447
however 10401
known 10328
states 10195
city 10180
united 9916
while 9913
where 9620
both 9600
number 9591
called 9290
made 8924
system 8794
through 8790
then 8789
being 8789
later 8767
including 8405
early 8403
under 8294
three 8270
state 8221
since 8136
often 8096
century 8056
part 7884
them 7861
english 7797
will 7792
people 7685
each 7603
well 7575
history 7520
because 7471
national 7466
university 7353
became 7332
same 7148
several 7139
example 6975
name 6926
government 6824
before 6762
year 6714
british 6670
against 6661
form 6600
group 6517
although 6377
found 6355
work 6341
second 

musical 1180
contain 1180
dark 1179
composer 1177
direction 1177
internal 1177
teams 1177
assembly 1176
course 1176
awards 1174
arts 1174
airport 1173
techniques 1172
world's 1170
parliament 1170
grand 1170
feature 1169
runs 1169
operating 1166
resulting 1165
recently 1165
forced 1164
reason 1163
equal 1161
1968 1159
annual 1158
fourth 1158
versions 1155
border 1154
intended 1153
management 1153
review 1153
potential 1152
motion 1152
administration 1152
remain 1152
note 1151
rise 1150
previously 1150
clear 1148
accepted 1148
website 1147
table 1145
formation 1145
prior 1144
signed 1142
effective 1140
carried 1140
critical 1139
frequently 1138
police 1137
worked 1137
plant 1136
materials 1136
initially 1135
episode 1135
daughter 1134
resources 1132
initial 1131
birth 1131
victory 1131
page 1130
distribution 1129
occur 1129
consists 1127
1960 1126
animal 1125
plan 1123
opened 1122
reduced 1120
things 1119
arabic 1118
becomes 1117
1980s 1117
presence 1115
models 1115
marriage 1113
buildin

castle 599
vast 599
revealed 599
agency 598
successfully 598
featuring 598
researchers 598
investment 598
philip 598
goes 597
persons 596
economics 595
rank 595
fine 595
brief 594
understood 594
measured 594
1936 593
landing 593
concluded 593
recognition 593
opera 593
holding 592
walter 592
seats 592
artillery 592
bishops 592
details 591
croatian 591
selection 591
brothers 591
patterns 591
occupied 591
glass 590
tourism 590
representation 590
civilization 589
calendar 589
essential 588
bands 588
1928 588
campus 588
franklin 587
inhabitants 587
1934 587
portugal 586
assigned 586
pakistan 586
corresponding 586
samuel 586
davis 586
angle 586
forming 585
facilities 585
younger 585
painter 584
dialects 584
documents 583
guard 583
it's 583
stand 583
taught 583
1914 582
coal 582
reality 581
twelve 581
northwest 581
armies 581
fort 580
acting 580
root 580
clock 579
williams 579
boys 579
broke 579
inner 579
entry 578
donald 578
nevertheless 578
sweden 578
engineer 577
confederate 577
attributed

extinction 389
rain 389
clinton 389
adams 389
linguistic 389
battles 389
vessels 389
tube 388
weapon 388
bones 388
1907 388
illinois 388
wine 388
occupation 388
survive 388
gospel 388
artistic 388
socialist 388
tissue 388
1904 388
longest 387
geographic 387
fluid 387
draft 387
communion 387
religions 387
influences 386
charges 386
losses 386
matches 386
gate 386
soft 386
defensive 386
alberta 385
exclusively 385
ministers 385
cannon 385
seconds 385
descendants 385
jobs 385
boat 384
benefits 384
ruling 384
lifetime 384
journey 384
manufacturers 384
cinema 384
unity 384
arizona 384
frederick 383
utility 383
psychological 383
greeks 383
syria 383
specified 383
afghan 383
continent 383
involve 383
singles 383
comprehensive 382
repeated 382
usual 382
parks 382
arguments 382
synthetic 382
municipal 382
perceived 381
thereby 381
supports 381
consequently 381
vowels 381
baptist 380
exports 380
well-known 380
debut 380
funds 380
bomb 380
cotton 380
isotopes 380
headed 379
northeast 379
lawrence

instances 299
botswana 299
victor 298
princeton 298
sydney 298
shadow 298
hugh 298
scheduled 298
peru 298
feed 297
pointed 297
racial 297
bicycle 297
capita 297
spears 297
dayton 297
equally 297
belfast 297
boron 297
tall 297
oxide 297
bolivia 297
smooth 296
adventures 296
discrimination 296
hearing 296
apostles 296
frac 296
likewise 296
primitive 296
seeds 296
girls 295
pronunciation 295
merchant 295
catherine 295
1893 295
songwriter 295
transmitted 295
jimmy 295
pool 295
agencies 294
demands 294
ward 294
overseas 294
twin 294
indicating 294
conventions 294
missing 294
priests 294
predicted 294
layers 294
corruption 294
pittsburgh 294
elementary 294
railways 294
sees 294
addressed 294
coined 294
reasoning 294
liberation 294
columns 293
singular 293
buddhism 293
symphony 293
dream 293
crimes 293
partnership 293
soundtrack 292
tested 292
aired 292
municipality 292
considering 292
regulations 292
seeing 292
phone 292
ascii 292
chains 292
advent 291
devoted 291
albania 291
symbolic 291
dr

...

anirul 3
faroula 3
noret 3
anti-gravity 3
blackmails 3
mcshane 3
dourif 3
frac{1}{x 3
frac{\part 3
f'(\mathbf{a})\mathbf{v 3
marwari 3
dramila 3
edma 3
mina's 3
grosset 3
emolument 3
oversimplification 3
non-cognitivism 3
schleiermacher 3
phillipson 3
schlick 3
20's 3
papillon 3
delmarva 3
ambrogio 3
single-field 3
televisa 3
tainting 3
enshrines 3
aggravation 3
mistrials 3
humiliate 3
corrosion-resistant 3
snopes 3
germanythe 3
powiat 3
wearer's 3
2-digit 3
drumline 3
pytho 3
fissure 3
trophonios 3
parnassos 3
fontenrose 3
linc-8 3
rsx-11 3
rsts 3
marvell 3
cardelli 3
lamport 3
6025 3
penises 3
grindhouse 3
disapproving 3
materialthe 3
yuppies 3
myeloid 3
quadruplex 3
wentworth-fitzwilliam 3
maleia 3
kyra 3
file:john 3
non-democratic 3
trotskyists 3
lipset 3
halperin 3
weatherford 3
monotonicity 3
quaver 3
halston 3
medallions 3
poppers 3
straight-ahead 3
strathspeys 3
gas-operated 3
cupcake 3
18033-3 3
bitstring 3
1/48 3
freeze-dried 3
phak 3
sesquilinear 3
compulsions 3
reverie 3
te

nairn 3
digne 3
baratier 3
meditate 3
khanjani 3
visigothorum 3
burgundionum 3
ripuarian 3
panopticon 3
sub-array 3
subarray 3
sub-intervals 3
server/provider 3
september–november 3
nakagawa's 3
end.the 3
close-quarters 3
river.on 3
panzers 3
hugging 3
pavlov's 3
eczema 3
göring's 3
tatsinskaya 3
shinshū 3
yogācāra 3
unsurprising 3
5-bromouracil 3
anquan 3
biscotti 3
raven's 3
birdwood 3
hepple 3
lecomber 3
thatcherism 3
redditch 3
ehrc 3
gemina 3
baptízomai 3
friberg 3
berardino 3
serato 3
raphoe 3
lachama 3
al-khader 3
muslim-christian 3
fenholt 3
i.r.s 3
kalsu 3
massdot 3
records.the 3
right_child 3
loop_top 3
flashheart 3
buffoon 3
incapability 3
covetous 3
boian 3
backgame 3
fensham 3
chiasm 3
roams 3
campier 3
bolland 3
48-page 3
batmen 3
colorization 3
buckner's 3
1901–1919 3
right-field 3
24–4 3
wire-to-wire 3
centerfielder 3
zill 3
south-eastwards 3
much-reduced 3
bohemia's 3
boihaemum 3
přemysl 3
taborites 3
weariness 3
blackburn's 3
nihal 3
mundie 3
worf 3
1821.the 3
d'erlon

tsiboe 3
re-recordings 3
crawler 3
berating 3
shut-down 3
bioaccumulate 3
247}_ 3
doldinger 3
co-conspirator 3
schreyer 3
sorbian 3
motorenwerke 3
aakirkeby 3
gudhjem 3
whole.in 3
ajdar-kadi 3
unhorsed 3
shakespearian 3
cruciferae 3
matthiola 3
sabr 3
4,270 3
13,941 3
d-nice 3
anagui's 3
vorkurs 3
hroðgar's 3
ohthere 3
prioritised 3
shippey 3
n-word 3
schismatrix 3
subdural 3
hijos 3
madrague 3
sweeney's 3
fingerpicks 3
carrasquel 3
cuban-born 3
left-field 3
bloop 3
wabc 3
juiciness 3
naphtali 3
cogan 3
powles 3
gallomania 3
oft-quoted 3
zebedee 3
pushbutton 3
höfner 3
palladino 3
percussively 3
myung 3
trainingthe 3
trichotomous 3
monoids 3
vinton's 3
blowpipe 3
shuttlecocks 3
artifices 3
visco 3
bodhimanda 3
paxman 3
non-controlling 3
buffalonian 3
96.8 3
lockport 3
self-hatred 3
arnould 3
1082 3
carreras 3
zealand-american 3
rossington 3
kiesewetter 3
elas 3
marinetti 3
kazakhstani 3
jouko 3
topp 3
yupanqui 3
illuminance 3
kerr's 3
ictis 3
outwit 3
one-to-many 3
mauceri 3
b,6a8 3
co

oxidization 3
9.05 3
tars 3
precombustion 3
feedstocks 3
lightbulb 3
21,500 3
1,203 3
97.8 3
85.6 3
75.4 3
64.6 3
50.6 3
vacuity-cold 3
etc.since 3
dampness 3
yin/yang 3
wheezing 3
herbalists 3
mulliken 3
hypervalent 3
subhumid 3
ndzwani 3
serendipitous 3
islands.on 3
iconi 3
2002.in 3
sixty-seven 3
1,146 3
african-arab 3
one-china 3
moheli 3
gravitate 3
crt-based 3
vitrified 3
shimmer 3
watson-crick 3
ovalene 3
ansatz 3
re-enact 3
defers 3
compilable 3
display.the 3
surtur 3
bitmaps 3
monergism 3
carnally 3
infralapsarianism 3
infra 3
saumur 3
surjections 3
carnem 3
remarrying 3
consciences 3
yamas 3
srimad 3
bhagavatam 3
manichaeans 3
arnett 3
azaria 3
dinant 3
hemphill 3
shemp 3
iannucci 3
knotts 3
1924–2006 3
lenk 3
raven-symoné 3
1917–1989 3
chesney 3
speight 3
bisection 3
arelate 3
reintegrate 3
cicero’s 3
minor's 3
morell 3
sum_{k=1}^\infty 3
peregrinus 3
pentameric 3
ehlers-danlos 3
heals 3
osteogenesis 3
glycosaminoglycans 3
silicones 3
gelatine 3
empathetic 3
unhurt 3
rampagi

islay 3
quetzalcoatl 3
astronomically 3
inscribing 3
moving-coil 3
d'arsonval 3
shunts 3
non-contact 3
brain-derived 3
azapirones 3
lactuca 3
once-daily 3
tetrahydrocannabinol 3
deluded 3
amberg 3
aspern-essling 3
baneful 3
reconsecrated 3
potentia 3
quatuor 3
1220s 3
pseudo-dionysius 3
quaestio 3
franciscana 3
solicitude 3
mononucleosis 3
alders 3
živojinović 3
mandola 3
spoonful 3
cochrane's 3
1990/1991 3
pauls 3
bzip2 3
ortygia 3
scholium 3
meleagrids 3
womanly 3
amarynthus 3
arktos 3
bolometer 3
oculocutaneous 3
lilady 3
masry 3
hala 3
forsan 3
prijs 3
fistfight 3
deren 3
medeiros 3
xmpp 3
user-interface 3
transferrable 3
threes 3
1079–1080 3
half-believers 3
islamisation 3
10.03 3
manastir 3
sigchi 3
oopsla 3
baptizes 3
stayer 3
klausen 3
belfort 3
black-tailed 3
priestcraft 3
waterboys 3
bushell 3
rigg 3
vocalizing 3
kasia 3
stritch 3
cryer 3
influencesthere 3
orgy 3
heitan 3
krarup 3
trite 3
role.in 3
melillo 3
suffragists 3
olshevsky 3
cross-referencing 3
4.4.8 3
heydrich 3
jel

Here you can see a part of an output on the sample dataset:

```
...
which 42198
this 38252
were 34403
...
```

Of course, the numbers may be different but not very much (the error about 2% will be accepted).